In [141]:
import pandas as pd
import nltk
import re
import numpy as np
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yolandaferreirofranchi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [142]:
#Json file option
#filejson = "C:/Users/danie/Desktop/bbc_news_list_uk.json"
#filecsv = r"C:\Users\danie\Documents\GitHub\Masters-Thesis\bbc_news_list_uk.csv"
filecsv = r"/Users/yolandaferreirofranchi/Documents/GitHub/Masters-Thesis/bbc_news_list_uk.csv"
article_df = pd.read_csv(filecsv)
article_df = article_df.assign(Article_Number=range(len(article_df)))
article_df = article_df.reset_index()
article_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15825 entries, 0 to 15824
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   index              15825 non-null  int64 
 1   tags               1 non-null      object
 2   title              15825 non-null  object
 3   news_post_date     15825 non-null  object
 4   raw_content        15468 non-null  object
 5   content            15468 non-null  object
 6   url                15825 non-null  object
 7   author             1453 non-null   object
 8   language           15825 non-null  object
 9   id                 15825 non-null  object
 10  region             15488 non-null  object
 11  short_description  15825 non-null  object
 12  category           15825 non-null  object
 13  crawled_at         15825 non-null  object
 14  Article_Number     15825 non-null  int64 
dtypes: int64(2), object(13)
memory usage: 1.8+ MB


In [143]:
#this code assumes the first four digits are the year. can be changed for last of middle
year = article_df['news_post_date'].str[:4]
article_df['year']=year

In [ ]:
#filename = 'articles.pkl'

#article_df = pd.read_pickle(filename)
#article_df = article_df.assign(Article_Number=range(len(article_df)))
#article_df = article_df.reset_index()
#article_df.info()

In [144]:
# tokenize sentences in an article 
import re

def split_sentences(article, article_id, year):
    pattern = r'(?<=[a-z0-9"]) *[.?!] *(?=[A-Z])'
    article = re.sub(pattern, r'\g<0> ', article)
    sentences = nltk.sent_tokenize(article)
    sentences_with_id = [(sentence, article_id, year) for sentence in sentences]
    return sentences_with_id

sentences_list = []

# add sentences to a new DF along with article ID 
for article, article_id, year in article_df[['content','Article_Number', 'year']].values:
    sentences = split_sentences(str(article), article_id, year)
    sentences_list.extend(sentences)

sentences_df = pd.DataFrame(sentences_list, columns= ['sentences', 'article_id', 'year'])

In [145]:
def pronoun_occurances(text):
    """ This function will count the number of female and male pronoun occurences in a given sentence."""
    pattern_m = r'(\s|^)(["\']?)(he|his|him|he\'s|hes)\2\b' #this regex will capture he/his as standalone words within a string but also at beginning of sentence
    matches_m = re.findall(pattern_m, text, re.IGNORECASE) #IGNORECASE is necessary to make sure that it picks up the pronouns at the beginning of a sentence
    pattern_f = r'(\s|^)(["\']?)(she|her|hers|shes|she\'s)\2\b'
    matches_f = re.findall(pattern_f, text, re.IGNORECASE)
    #pattern_n = r'(\s|^|")(["\']?)(they|them|their|theirs|their\'s)\2\b' #
    #matches_n = re.findall(pattern_n, text, re.IGNORECASE)
    count_m = len(matches_m)
    count_f = len(matches_f)
    #count_n = len(matches_n)
    #count_u = count_n + count_f + count_m
    return count_f, count_m #, count_n, count_u

reporting_words = ["acknowledge", " add", " address", " admit", " announce", " argue", " believe", " claim", " conclude", " confirm", " continue", " declare", " describe", " ensure", " estimate", " explain", " find", " indicate", " inform", " insist", " note", " point", " predict", " provide", " release", " reply", " report", " respond", " say", " state", " suggest", " tell", " testify", " think", " tweet", " warn", " write"]

 If sent contains a reporting word then
    look to the word before for a name or a pronoun
        If found and a name find gender 
            once gender is found clasify the sentence
        if found a pronoun then classify sentence
        else   
            look after the word for the same and follow the same pattern
                else follow the rest of the pattern
Count the number of male, female, and neutral pronouns

*****Look to see if the sentence contains a proper noun and add the gender to the ocunt
    I think this should be added to the preprocessing because we are already looking up the POS tags so it will use less bullshit

In [147]:
#create a variable applying the function of pronoun occurence
sent = sentences_df['sentences'].apply(pronoun_occurances)
# Create two new columns in sentences DF from the tuple output in "sent"
sentences_df['female_count'] = [x[0] for x in sent]
sentences_df['male_count']= [x[1] for x in sent]
#sentences_df['neutral_count']= [x[2] for x in sent]
#sentences_df['u_count']= [x[3] for x in sent]

#Bug is fixed and now it counts properly
sentences_df

,sentences,article_id,year,female_count,male_count
0,The heroin substitute methadone can be used as...,0,2010,0,0
1,Earlier this year a debate broke out in Scotla...,0,2010,0,0
2,"But a group of 40 specialists, including unive...",0,2010,0,0
3,So what do recovering addicts think?,0,2010,0,0
4,Chris used methadone for five years to help we...,0,2010,0,3
...,...,...,...,...,...
225660,Similar moves in Europe have sparked cries of ...,15824,2010,0,0
225661,The lead in the Daily Mail is a claim that cou...,15824,2010,0,0
225662,It says authorities are using the information ...,15824,2010,0,0
225663,The Daily Express leads with a warning from US...,15824,2010,0,0


In [148]:
def compare_count(male_col, female_col): 
    """This function compares the count of female to male pronouns. It will output "1" if male count bigger
    than female count, "neutral" if the count is equal, and "female" if there is a higher female count. 
    The function returns strings because we need categorical variables for log reg to run"""
    if female_col > male_col:
        return "0"
    elif male_col > female_col:
        return "1"
    else: 
        return None
sentences_df['col_type'] = sentences_df.apply(lambda row: compare_count(row['male_count'], row['female_count']),axis=1)
sentences_df

,sentences,article_id,year,female_count,male_count,col_type
0,The heroin substitute methadone can be used as...,0,2010,0,0,None
1,Earlier this year a debate broke out in Scotla...,0,2010,0,0,None
2,"But a group of 40 specialists, including unive...",0,2010,0,0,None
3,So what do recovering addicts think?,0,2010,0,0,None
4,Chris used methadone for five years to help we...,0,2010,0,3,1
...,...,...,...,...,...,...
225660,Similar moves in Europe have sparked cries of ...,15824,2010,0,0,None
225661,The lead in the Daily Mail is a claim that cou...,15824,2010,0,0,None
225662,It says authorities are using the information ...,15824,2010,0,0,None
225663,The Daily Express leads with a warning from US...,15824,2010,0,0,None


In [159]:
#remove columns with "None" in the col_type 
sentences_df = sentences_df[sentences_df["col_type"].notnull()]
#sentences_df

In [149]:
sentences_df.sort_values("female_count", ascending = False)

,sentences,article_id,year,female_count,male_count,col_type
46585,"""She left her job, she left someone in her fla...",2967,2010,7,1,0
175442,Mrs Jones previously admitted failing to notif...,12116,2010,7,1,0
60832,She also said that she had cut her mother's na...,4004,2010,7,0,0
23016,"In her appeal, she claimed she only pleaded gu...",1339,2010,7,1,0
9083,The charge alleges Mr McLeod failed to observe...,466,2010,6,0,0
...,...,...,...,...,...,...
77381,"""""I, Shahram Amiri, a citizen of the Islamic R...",5146,2010,0,0,None
77382,I am free here and I assure everyone that I am...,5146,2010,0,0,None
77383,My purpose in today's conversation is to put a...,5146,2010,0,0,None
77384,I am Iranian and I have not taken any steps ag...,5146,2010,0,0,None


0 - female 
1 - male

**Building the Classifier**

**Logistics Regression Classifier**

In [150]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 
from nltk.corpus import stopwords

In [151]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [152]:
#nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from genderize import Genderize

#Sentence Encoding
def tidy_text(sentence, remove_stopwords = True):

    sentence = re.sub(r'https?:\/\/.*[\r\n]*', '', sentence)
    sentence = re.sub(r'\<a href', ' ', sentence)
    sentence = re.sub(r'&amp;', '', sentence) 
    sentence = re.sub("\d+", "", sentence)
    
    #changed the number detection code
    sentence = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', sentence)
    sentence = re.sub(r'<br />', ' ', sentence)
    sentence = re.sub(r'\'', ' ', sentence)

    # Tokenize each word
    sentence =  nltk.WordPunctTokenizer().tokenize(sentence)

    #! I think this is where the gender words detection should go, because we can do it after tokenization and then it's more efficiant
    #we also have a bit of a plural problem. we need to take care of. Also the words need to be removed--maybe at the end? 
    #so this function here will return a male, female count 

    nltk.tag.pos_tag(sentence)
    tagged_sentence = nltk.tag.pos_tag(sentence)
    lemma = nltk.stem.WordNetLemmatizer()


    pn_tags = ('NNP', 'NNPS')
    new_words = []

    for word, tag in tagged_sentence: 
        if tag not in pn_tags: 
            if tag.startswith("V"):
                lemmas = lemma.lemmatize(word, "v")
            else: 
                lemmas = lemma.lemmatize(word)
            
            new_words.append((lemmas))
        #!# We need to add an else statement here to process PN words, there could be some trickyness here but it should be fine. 
        #else:
            #gender = Genderize().get(word)
            #reteurns 'male' and 'female' 
                #if gender = 'male'
                 #   malelistcount add 1
                #if gender = 'female'
                 #   femalelistcount add 1


    sentence = new_words

    # Expand contractions
    if True:
        new_text = []
        for word in sentence:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
    
    
    # remove stopwords
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        sentence = [w for w in sentence if not w in stops]

    #!  here we check for words from the gender words and we ensure they are removed from the tokens. this will ensure we have 0 leakage
    
    return sentence

#text = "I ate a sandwich and it was very tasty"



In [153]:
sentences_df['encoded_sentences'] = sentences_df['sentences'].apply(tidy_text)

In [160]:
sentences_df

,sentences,article_id,year,female_count,male_count,col_type,encoded_sentences
4,Chris used methadone for five years to help we...,0,2010,0,3,1,"[use, methadone, five, year, help, wean, heroi..."
6,"""I heard about Ratho Hall, and it's given me t...",0,2010,0,1,1,"[I, hear, give, wake, call, I, need, say]"
7,"Chris was prescribed methadone, a synthetic op...",0,2010,0,2,1,"[prescribe, methadone, synthetic, opiate, take..."
12,"In this, the Community Safety Minister, Fergus...",0,2010,0,1,1,"[In, explain, want, see, new, vision, drug, tr..."
14,"Chris, who started to use heroin when he was 1...",0,2010,0,2,1,"[start, use, heroin, talk, addiction, When, he..."
...,...,...,...,...,...,...,...
225598,"""For Diego, it is good for him to see the norm...",15819,2010,0,2,1,"[For, good, see, normal, rearing, process, wit..."
225601,The new monkey will stay at the zoo until he r...,15819,2010,0,3,1,"[The, new, monkey, stay, zoo, reach, sexual, m..."
225605,He died in hospital.,15820,2010,0,1,1,"[He, die, hospital]"
225645,"He said: ""Local authorities need to recognise ...",15823,2010,0,1,1,"[He, say, authority, need, recognise, regional..."


In [ ]:
#pd.get_dummies(sentences_df.year)
#rated_dummies = pd.get_dummies((sentences_df).year)
#sentences_df = pd.concat([sentences_df, rated_dummies], axis=1)

In [ ]:
#X = sentences_df[['encoded_sentences', '2010', '2012']]
#y = sentences_df["col_type"]
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [164]:
X = sentences_df['encoded_sentences']
y = sentences_df['col_type']
X_train, X_test, y_train, y_test = train_test_split(X_t, y, test_size=0.2, random_state=42)

In [156]:
#vectorizer = TfidfVectorizer(max_features= 1000, lowercase=False, tokenizer=False)
def fake(token):
    return token

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=fake,
    preprocessor=fake,
    token_pattern=None)  

X_t = tfidf.fit_transform(X)
#X_train = tfidf.fit_transform(X_train)
#X_test = tfidf.fit_transform(X_test)

**Binary Logistics Regression**

In [ ]:
#GridSearchCV Packages
from sklearn.model_selection import GridSearchCV
import warnings 
warnings.filterwarnings("ignore")


# parameter grid
parameters = {"penalty": ['l1','l2'], 
              "C": np.logspace(-3,3,7),
              "solver": ['newton-cg', 'lbfgs', 'liblinear'],
}

In [ ]:
#GridSearchCV
logreg = LogisticRegression()
clf_logreg = GridSearchCV(logreg, 
                          param_grid = parameters, 
                          scoring = "accuracy", 
                          cv = 10)

In [ ]:
clf_logreg.fit(X_train, y_train)

In [165]:
#finetuned model with best hyperparameters 
logreg1 = LogisticRegression(C=0.1, penalty = "l2", solver = "liblinear", max_iter = 5000)

LogisticRegression(C = 0.1, penalty = "l2", solver = "liblinear", max_iter = 5000)

In [166]:
fitted_logreg = logreg1.fit(X_train, y_train)
fitted_logreg

LogisticRegression(max_iter=5000, solver='liblinear')

In [174]:
prediction = fitted_logreg.predict(X_test)

In [169]:
print("Accuracy:",fitted_logreg.score(X_test, y_test))

Accuracy: 0.8582270795385549


L2 is better to prevent overfitting than L1 because it punishes errors more (squares them as opposed to taking the absolute value). GridSearchCV said either L1 or L2 were equally as good though. 

In [173]:
coefs = fitted_logreg.coef_[0]

#male is 1 and female is 0

sorted_coef = sorted((zip(tfidf.get_feature_names(), coefs)), key = lambda x: x[1], reverse=True)

high_coef = sorted_coef[:10]
low_coef = sorted_coef[-10:]

print("highest coefs")
for i in high_coef: 
    print(i)

print("lowest coefs")
for i in low_coef: 
    print(i)

highest coefs
('He', 15.727315039161565)
('His', 7.467566777876769)
('man', 4.3397941552226635)
('wife', 3.8827756924014567)
('soldier', 2.646811096063486)
('shoot', 2.474074718318458)
('general', 1.9728945249627372)
('boy', 1.9528321539953917)
('player', 1.9137759838719437)
('football', 1.8083109516570461)
lowest coefs
('female', -2.9502895317018543)
('boyfriend', -3.245074680432046)
('mother', -3.4304259369500096)
('daughter', -3.621111448681256)
('actress', -4.277934839317325)
('girl', -5.307364280267746)
('husband', -6.295752606185669)
('woman', -7.245390395868678)
('Her', -9.577062358981655)
('She', -18.650879417534796)


Interestingly, male coefficient for neutral is higher than the female. Otherwise, as expected, the coefficients for female and male each are correspondingly high for each gender. 

In [177]:
print(metrics.accuracy_score(y_test, prediction)) #accuracy 
print(metrics.confusion_matrix(y_test, prediction)) #confusion matrix
print(metrics.classification_report(y_test, prediction))

0.8582270795385549
[[ 816 1312]
 [  89 7665]]
              precision    recall  f1-score   support

           0       0.90      0.38      0.54      2128
           1       0.85      0.99      0.92      7754

    accuracy                           0.86      9882
   macro avg       0.88      0.69      0.73      9882
weighted avg       0.86      0.86      0.83      9882

